In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import PIL
import pathlib

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Dropout, Flatten,Activation, BatchNormalization,MaxPooling2D
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [ ]:
data_dir = pathlib.Path("/content/drive/MyDrive/Transfer_Learing/dogs_cats/PetImages")

NameError: ignored

In [ ]:
dogs = list(data_dir.glob('Dog/*'))

In [ ]:
cats = list(data_dir.glob('Cat/*'))

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))

In [ ]:
image_count

In [ ]:
pet_images_dict = {
    'cats': list(data_dir.glob('Cat/*')),
    'dogs': list(data_dir.glob('Dog/*')),
}

pet_labels_dict = {
    'cats': 0,
    'dogs': 1,
}

In [ ]:
IMAGE_WIDTH=150
IMAGE_HEIGHT=150
X, Y = [], []

for pet_name, images in pet_images_dict.items():
    print(pet_name)
    for image in images:
        img = cv2.imread(str(image))
        if isinstance(img,type(None)): 
            continue
            
        elif ((img.shape[0] >= IMAGE_HEIGHT) and  (img.shape[1] >=IMAGE_WIDTH)):
            resized_img = cv2.resize(img,(IMAGE_WIDTH,IMAGE_HEIGHT))
            X.append(resized_img)
            Y.append(pet_labels_dict[pet_name])
        else:
            continue

In [ ]:
X = np.array(X)
Y = np.array(Y)

In [ ]:
X= X/255

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, random_state=42)

In [ ]:
vgg19 = tf.keras.applications.VGG19(
    include_top = False, 
    weights = 'imagenet', 
    input_tensor = None,
    input_shape = (150,150,3), 
    pooling = None, 
    classes = 1000
)

In [ ]:
vgg19.trainable = False

In [ ]:
model_vgg = tf.keras.models.Sequential([
    
    vgg19,
    tf.keras.layers.Conv2D(128, kernel_size = (3, 3), padding = 'same'),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),# modifying final layers of VGG-19
    tf.keras.layers.Conv2D(64, kernel_size = (3, 3), padding = 'same'),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100),
    tf.keras.layers.PReLU(alpha_initializer='zeros'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
import tensorflow_addons as tfa

In [ ]:
metrics = ['accuracy']

In [ ]:
model_vgg.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = metrics)
model_vgg.summary()

In [ ]:
history_vgg = model_vgg.fit(
    X_train,y_train,
    epochs = 30,
    validation_data = (X_val, Y_val),
    batch_size=32,
    verbose = 1)